# Normalize LIMS columns

Normalize the columns for the eODP CSVs.

In [1]:
import sys
sys.path.append('../../../')
from pathlib import Path

import pandas as pd
import shutil

from config import CLEAN_DATA_DIR, OUTPUT_DIR
from scripts.normalize_data import (
    normalize_sample_col, 
    normalize_expedition_section_cols, 
    csv_cleanup,
    update_metadata,
    fetch_unique_column_names,
    append_set,
    filter_existing_set,
    normalize_columns,
    add_missing_columns,
    extract_sample_parts,
    create_sample_name_for_row
)

In [2]:
base_dir = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 



In [3]:
def log_df(df, row_count=5):
    print(df.shape)
    return df.head(row_count)


## check basic columns

check if files have basic columns

In [33]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False


In [34]:
bad_files = set()
for file in metadata['path']:     
    
    path = base_dir/file
    df = pd.read_csv(path, dtype=str, nrows=0)
    cols = df.columns
    if ('Sample' in cols or 'Label ID' in cols) or (
        'Exp' in cols
        and 'Hole' in cols 
        and 'Site' in cols 
        and 'Core' in cols 
        and 'Type' in cols 
        and 'Section' in cols
        ):
        pass
    else:
        bad_files.add(path)
        

        
len(bad_files)

0

In [35]:
bad_files

set()

## Normalize expedition..section columns 

Read each Lithology CSV to check if expedition..section columns exist. Overwrite existing Lithology CSV if columns need to be added.

In [4]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False


In [5]:
metadata.shape

(1253, 9)

In [7]:
def process_filename(file):
    path = base_dir/file
    content = pd.read_csv(path, dtype=str)
    
    original_cols = content.columns
    try:
        content = normalize_expedition_section_cols(content)
    except ValueError as err:
        print(file, err)
    changed = list(original_cols) != list(content.columns)

        
    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [process_filename(file) for file in metadata['path']] 


In [10]:
dict = {"add_expedition_section_cols": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False


In [11]:
new_metadata.to_csv(metadata_file, index=False)

## Normalize Sample column

Read each  CSV to check if Sample column needs to be updated. Change 'Label ID' to 'Sample'. Add 'Sample' if no 'Sample' or 'Label ID' based on expedition...section columns. Overwrite existing Lithology CSV if Sample column is updated.

In [12]:
metadata = pd.read_csv(metadata_file)
metadata.head(2)

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False


In [13]:
metadata.shape

(1253, 10)

In [14]:
def process_filename(file):
    path = base_dir/file
    content = pd.read_csv(path, dtype=str)
    
    original_cols = content.columns
    normalize_sample_col(content)
    
    changed = list(original_cols) != list(content.columns)
    
    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [process_filename(file) for file in metadata['path']] 

In [17]:
dict = {"update_sample_col": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True


In [18]:
new_metadata.to_csv(metadata_file, index=False)

## Normalize Top and Bottom columns

Normalize all the Top, Top Depth, Bottom, and Bottom Depth column to have the same names.

In [19]:
metadata = pd.read_csv(metadata_file)
log_df(metadata)

(1253, 11)


,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True


### Get top bottom columns

In [20]:
columns_all = set()

res=[fetch_unique_column_names(base_dir/file, columns_all) for file in metadata['path']] 

In [21]:
top_all = set()
top = set()
top_depth = set()

bottom_all = set()
bottom = set()
bottom_depth = set()


append_set(top_all, r".*?\btop\b.*?", columns_all)
append_set(top_depth, r"top depth", columns_all)
append_set(top, r"top ?\[", columns_all)

append_set(bottom_all, r".*?\bbottom\b.*?", columns_all)
append_set(bottom_depth, r"bottom depth", columns_all)
append_set(bottom, r"bottom ?\[", columns_all)

In [22]:
top_all

{'Original Top Depth (m)',
 'Top Depth [m]',
 'Top Depth[m] [m]',
 'Top Offset (cm) on Parent Sample',
 'Top [cm]',
 'Top depth [m]',
 'Top[cm] [cm]'}

In [23]:
top

{'Top [cm]', 'Top[cm] [cm]'}

In [24]:
top_depth

{'Top Depth [m]', 'Top Depth[m] [m]', 'Top depth [m]'}

In [25]:
bottom_all

{'Bottom Depth [m]',
 'Bottom Depth[m] [m]',
 'Bottom Offset (cm) on Parent Sample',
 'Bottom [cm]',
 'Bottom depth [m]',
 'Bottom[cm] [cm]',
 'Original Bottom Depth (m)'}

In [26]:
bottom

{'Bottom [cm]', 'Bottom[cm] [cm]'}

In [27]:
bottom_depth

{'Bottom Depth [m]', 'Bottom Depth[m] [m]', 'Bottom depth [m]'}

### Normalize top bottom columns

In [28]:
dict.fromkeys(top, 'Top [cm]')

{'Top[cm] [cm]': 'Top [cm]', 'Top [cm]': 'Top [cm]'}

In [29]:
path = base_dir/'LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv'
content = pd.read_csv(path, dtype=str)
normalize_columns(content, dict.fromkeys(top, 'Top [cm]'))

In [30]:
def normalize_top_bottom(file):
    path = base_dir/file
    content = pd.read_csv(path, dtype=str)
    columns = list(content.columns)
    
    normalize_columns(content, dict.fromkeys(top, 'Top [cm]'))
    normalize_columns(content, dict.fromkeys(bottom, 'Bottom [cm]'))
    normalize_columns(content, dict.fromkeys(top_depth, 'Top Depth [m]'))
    normalize_columns(content, dict.fromkeys(bottom_depth, 'Bottom Depth [m]'))
    
    normalized_cols = list(content.columns)
    changed = columns != normalized_cols
    
    if changed:
        content.columns = normalized_cols
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed

change_columns = [normalize_top_bottom(file) for file in metadata['path']] 

In [31]:
dict = {"update_top_bottom": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False


In [32]:
new_metadata.to_csv(metadata_file, index=False)

## Add missing columns

In [33]:
metadata = pd.read_csv(metadata_file)

In [34]:
normalized_columns = [
    'Top [cm]',
    'Bottom [cm]',
    'Top Depth [m]',
    'Bottom Depth [m]', 
    'Sample',
    'Exp',
    'Site',
    'Hole',
    'Core',
    'Type',
    'Section',
    'A/W'
]

In [35]:
change_columns = [add_missing_columns(base_dir/file, normalized_columns) for file in metadata['path']] 

In [36]:
dict = {"add_missing_cols": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False


In [37]:
new_metadata.to_csv(metadata_file, index=False)